In [16]:
#可以支援多種文字、二進位檔案與資料庫的資料載入，常見的 txt、csv、excel 試算表、MySQL 或 PostgreSQL 都難不倒，以及處理網頁表格問題等
import pandas as pd
#建立各種 HTTP 請求，從網頁伺服器上取得想要的資料。
import requests
#requests-html具備HTTP庫把網頁下載下來與解析。此項為直接使用html
from requests_html import HTML
#對文件做正規表示法
import re
import time
#在 xlsxwriter替换掉非法字符，省去了手工操作。
import xlsxwriter

# 檢視資料結構
ID = '232174987'
#使用dcard的api爬網
url = 'https://www.dcard.tw/_api/posts/' + ID
# 透過request套件抓下這個網址的資料
requ = requests.get(url)
# 初步檢視抓到的資料結構
requ.json()

{'id': 232174987,
 'title': '＃問＃健身手套',
 'content': '想請問各位有在健身的女孩們\n有沒有在戴手套練的跟推薦品牌？\n我徒手做到手都快要長繭了好難摸😭😭',
 'excerpt': '想請問各位有在健身的女孩們，有沒有在戴手套練的跟推薦品牌？我徒手做到手都快要長繭了好難摸',
 'anonymousSchool': False,
 'anonymousDepartment': True,
 'pinned': False,
 'forumId': '9b5213d6-4427-4a3c-b788-efa19493631c',
 'replyId': None,
 'createdAt': '2019-09-26T13:50:08.911Z',
 'updatedAt': '2019-09-26T13:50:08.911Z',
 'commentCount': 16,
 'likeCount': 7,
 'tags': [],
 'topics': ['重訓', '健身'],
 'supportedReactions': None,
 'withNickname': False,
 'reportReason': '',
 'hiddenByAuthor': False,
 'meta': {'layout': 'classic'},
 'forumName': '健身',
 'forumAlias': 'fitness',
 'school': '中國醫藥大學',
 'replyTitle': None,
 'gender': 'F',
 'personaSubscriptable': None,
 'reactions': [{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03', 'count': 7}],
 'hidden': False,
 'customStyle': None,
 'layout': 'classic',
 'withImages': False,
 'withVideos': False,
 'media': [],
 'reportReasonText': '',
 'mediaMeta': [],
 'postAvatar': ''}

In [17]:
# 將抓下來的資料轉為DataFrame
ID = '232174987'
url = url = 'https://www.dcard.tw/_api/posts/' + ID
requ = requests.get(url)
rejs = requ.json()
pd.DataFrame(
    data=
    [{'ID':rejs['id'],
      'title':rejs['title'],
      'content':rejs['content'],
      'excerpt':rejs['excerpt'],
      'createdAt':rejs['createdAt'],
      'updatedAt':rejs['updatedAt'],
      'commentCount':rejs['commentCount'],
      'forumName':rejs['forumName'],
      'forumAlias':rejs['forumAlias'],
      'gender':rejs['gender'],
      'likeCount':rejs['likeCount'],
      'reactions':rejs['reactions'],
      'topics':rejs['topics']}],
    columns=['ID','title','content','excerpt','createdAt','updatedAt','commentCount','forumName','forumAlias','gender','likeCount','reactions','topics'])

,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,232174987,＃問＃健身手套,想請問各位有在健身的女孩們\n有沒有在戴手套練的跟推薦品牌？\n我徒手做到手都快要長繭了好難摸😭😭,想請問各位有在健身的女孩們，有沒有在戴手套練的跟推薦品牌？我徒手做到手都快要長繭了好難摸,2019-09-26T13:50:08.911Z,2019-09-26T13:50:08.911Z,16,健身,fitness,F,7,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 健身]"


In [18]:
# 撰寫簡單的函數，透過輸入文章ID，就輸出文章的資料
def Crawl(ID):
    link = 'https://www.dcard.tw/_api/posts/' + str(ID)
    requ = requests.get(link)
    try:
        rejs = requ.json()
        return(pd.DataFrame(
            data=
            [{'ID':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt':rejs['updatedAt'],
              'commentCount':rejs['commentCount'],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'likeCount':rejs['likeCount'],
              'reactions':rejs['reactions'],
              'topics':rejs['topics']}],
            columns=['ID','title','content','excerpt','createdAt','updatedAt','commentCount','forumName','forumAlias','gender','likeCount','reactions','topics']))
    except:
        print('不是json')

# 嘗試使用撰寫出的函數，抓取編號231030181的文章
Crawl(232174987)

,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,232174987,＃問＃健身手套,想請問各位有在健身的女孩們\n有沒有在戴手套練的跟推薦品牌？\n我徒手做到手都快要長繭了好難摸😭😭,想請問各位有在健身的女孩們，有沒有在戴手套練的跟推薦品牌？我徒手做到手都快要長繭了好難摸,2019-09-26T13:50:08.911Z,2019-09-26T13:50:08.911Z,16,健身,fitness,F,7,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 健身]"


In [19]:
# 一次讀取100篇最熱門的文章，dcard的api最多抓取100篇。
url = 'http://dcard.tw/_api/forums/fitness/posts?popular=false&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    #append() 方法用于在列表末尾添加新的对象。
    df = df.append(Crawl(rejs[i]['id']),ignore_index=True)
#.shape功能是查看矩陣或者數組的維數。
print(df.shape)
df

(100, 13)


,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,232227133,如何練成歐美肥臀？,用台灣的健身網紅Ashlee與Jessica舉例\n我一直好奇為什麼兩位體型不太一樣\n我想...,用台灣的健身網紅Ashlee與Jessica舉例，我一直好奇為什麼兩位體型不太一樣，我想過以...,2019-10-04T03:25:25.304Z,2019-10-04T03:25:25.304Z,5,健身,fitness,F,1,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 肌肉, 體脂, 飲食, 健身]"
1,232227083,啞鈴划船問題,各位巨巨好，想問在做單手啞鈴划船時，採用弓箭步，將重量重心放在前側腿及屁股，背是平的，疑問在...,各位巨巨好，想問在做單手啞鈴划船時，採用弓箭步，將重量重心放在前側腿及屁股，背是平的，疑問在...,2019-10-04T03:13:28.850Z,2019-10-04T03:13:56.019Z,2,健身,fitness,M,1,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[重訓]
2,232226994,老人根本是健身房的毒瘤,最近公立運動中心健身房越來越多老人\n尤其是非熱門時段\n他們總是插銷插最輕\n用完全錯誤的...,最近公立運動中心健身房越來越多老人，尤其是非熱門時段，他們總是插銷插最輕，用完全錯誤的姿勢做...,2019-10-04T02:54:57.521Z,2019-10-04T02:54:57.521Z,8,健身,fitness,M,5,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[健身房, 健身器材]"
3,232226923,inbody 名詞/測量項目請益,https://i.imgur.com/12XHp7o.jpg\n\n各位巨巨好，小的新手健...,各位巨巨好，小的新手健人，接觸約2個半月，第一次量inbody，因為一些緣故所以沒有專業的人...,2019-10-04T02:36:10.001Z,2019-10-04T02:37:20.724Z,0,健身,fitness,M,0,[],"[Inbody, 健身]"
4,232226709,睡眠很重要 不要沒睡飽,奉勸各位如果前一天沒有睡好\n就盡量不要去做重訓\n當然這是我個人的經驗談\n不一定每個人都...,奉勸各位如果前一天沒有睡好，就盡量不要去做重訓，當然這是我個人的經驗談，不一定每個人都適用 ...,2019-10-04T01:48:18.828Z,2019-10-04T01:50:23.495Z,5,健身,fitness,M,5,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 健身房]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,232191176,TANITA體脂計推薦嗎?(BF047 、UM051),最近想要買個體脂計 來記錄一下體重、體脂變化\n看到這兩款都特價 ，滿便宜的\n想問一下有人...,最近想要買個體脂計 來記錄一下體重、體脂變化，看到這兩款都特價 ，滿便宜的，想問一下有人買過...,2019-09-29T06:14:24.215Z,2019-09-29T06:14:24.215Z,5,健身,fitness,M,6,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[健身, 體脂, 體脂計, 體重, 減肥]"
96,232190343,光頭網紅亂開藥單,https://youtu.be/OkjxklixR3\n用藥我不懂，網紅光頭的我懂，中風的...,用藥我不懂，網紅光頭的我懂，中風的我也懂，各位粉絲請增大雙眼吧！，連結好像有問題，補上圖片自...,2019-09-29T03:11:54.879Z,2019-09-29T03:19:45.338Z,42,健身,fitness,M,43,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[用藥, 網紅]"
97,232190329,增肌體重突然下降,各位好，我現在正在增肌的第一個半月\n已經卡在75~75.5公斤約兩周了（一直上下漂浮\n然...,各位好，我現在正在增肌的第一個半月，已經卡在75~75.5公斤約兩周了（一直上下漂浮，然而這...,2019-09-29T03:08:22.985Z,2019-09-29T03:08:22.985Z,16,健身,fitness,M,14,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[增肌, 重訓, 肌肉, 飲食]"
98,232190271,想問你們一週的訓練菜單是都怎麼排,我是暑假才開始健身的一開始就是練我想要的胸肩背二頭三頭，但是最近學了三項，而且開學了訓練項目...,我是暑假才開始健身的一開始就是練我想要的胸肩背二頭三頭，但是最近學了三項，而且開學了訓練項目...,2019-09-29T02:51:42.876Z,2019-09-29T02:51:42.876Z,22,健身,fitness,M,15,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 增肌, 肌肉, 健身房, 健身教練]"


In [34]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
try:
    for j in range(110):
        #.tail(1)顯示最後一筆
        last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
        #由於API限制最多載入100篇文章，如果我們想要爬更多資料，可以透過before參數與迴圈進行!
        url = 'https://www.dcard.tw/_api/forums/fitness/posts?popular=false&limit=100&before=' + last
        resq = requests.get(url)
        try:
            rejs = resq.json()
            time.sleep(5)
            for i in range(len(rejs)):
                #合併資料
                print(j)
                print(rejs[i]['title'])
                df = df.append(Crawl(rejs[i]['id']), ignore_index=True)
                time.sleep(5)
        except:
            print('不是json')
except KeyError:
    print('已經沒有了')
print(df.shape)
df

0
#問 拉單槓的訓練方式
0
#問 現在還有練腿上半身會長比較快的說法嗎？
0
#閒聊 運動怪習慣
0
手肘發出聲音?
0
#問 淡水健身工廠推薦教練
0
#瘦身#運動#都沒有用 我的體重還是一樣......
0
#問 碳水循環
0
關於肌肉痠痛的問題
0
雙腿伸屈的最後一下
0
吸套起飛了
0
#分享 3分鐘內消耗50大卡的訓練 (中文字幕)
0
推薦優格
0
#問 關於deload
0
左右不平衡
0
#問 油脂不足
0
#請益 減脂以及健身
0
#詢問 划船姿勢 腹外斜肌酸痛
0
#問 間歇性斷食
0
關於重訓天數
0
#討論 跟自由重量動作很像的機械式器材
0
健身是興趣會想讓它成為工作嗎？
0
請問高蛋白加什麼會比較好喝
0
#影 YO!PEACE 館長對健身網紅看法
0
間歇斷食小疑問
0
INBODY
0
#分享 適合下背痛的7種核心訓練動作 (中文字幕)
0
#分享 快速燃燒大腿外側的訓練 (中文字幕)
0
cellmass肌酸
0
健身穿什麼顏色壯
0
變胖？變瘦？
0
肉鹼有用嗎？
0
請益 #deload
0
#消息 豐原要開中力嘍
0
健身問題 解答
0
#問 減脂飲食 碳水化合物
0
#分享 人生第一次減脂週期
0
#問 為了吃大餐而健身?
0
吃肌酸
0
#分享 Jesse與職業摔角手的比賽 (中文字幕)
0
練後肌肉痠痛
0
＃問＃減脂＃請益
0
這樣瘦對嗎？
0
#分享 低強度恆速有氧運動的效果 | Thomas DeLauer的健康秘訣 (中文字幕)
0
#問 關於南瓜的熱量請益
0
徵求健身伙伴～
0
半年了 從瘦皮猴到正常人
0
#問 運動後吃的迷思
0
超實用極限換算
0
#問 增肌應該什麼時候停止
0
瘦子的增重速度
0
胸肌問題
0
給館長很多個讚！👍🏻👍🏼👍
0
生理期想吃甜食
0
和美、台中健身選擇
0
關於補劑 ZMA
0
#成果詢問 #一個月掉4%脂肪增加肌肉2kg
0
#問 到底能不能繼續做深蹲？（膝蓋有喀聲）
0
請問北高有短期（月、季）的健身房嗎？
0
#問 深蹲裝備
0
#問 去年九月開始健身 曾練過柔道 求帶路
0
#分享 健身後別做的事情
0
蔬菜店問題
0
人魚線跟鯊魚肌怎麼練？
0
#健身房拉筋
0
淡水健工求夥伴
0
分享我的新手健身菜單
0
台中健身房
0
徵求健身朋友
0
健美式臥推 肩胛夾

#飛輪 同學現在是逆風
5
#請益 練的順序
5
請問我該如何吃比較好?
5
#問 健身房選擇及評價(本人住士林)
5
#分享 槓鈴的腹部訓練 (中文字幕)
5
#分享 讓你有動力健身的6大秘訣 (中文字幕)
5
#健身室 #健身 #方便 #台北市 #新北市 #台中市
5
#黑特 world gym
5
#健身 Roadblock 菜單與課表
5
想練二、三頭肌？
5
#討論 再討論一些健身房的惡習
5
這樣真的能減肥嗎？
5
#影片 深蹲後下背會酸
5
#問 #重訓 重量應該遞增還是應該遞減
5
#分享 初學者腹部訓練的錯誤 (中文字幕)
5
#討論   討論澱粉 討論白飯 飲食迷思
5
剛練而已
5
#重訓 world gym 健身工廠 選擇
5
肌肉量好重怎麼辦...
5
#問 #健身 屏東健身房和健身知識詢問
5
巔峰期求方法
5
#分享 適合男性的增肌守則 (中文字幕)
5
#梗圖 最近的標籤梗圖燒到健身界了
5
到底怎麼樣叫作練太多？
5
#問 女生三項
5
#問 家用飛輪有推薦的嗎？
5
#問 #Polar H10心跳帶#Polar beat
5
#問 健身腰帶選擇 健美用
5
#問 睡眠嚴重不足也會逼自己去健身嗎?
5
#問 有沒有因為坐墊而疼痛的掛
5
#問 體重體脂
5
#分享 #重訓 #減脂 #Jack the ball 我的16週減脂(6)
5
#問 關於飲食和體重
5
#問 inbody這樣要先減脂還增肌
5
#換氣症候群
5
關於壓縮褲 #問 #重訓 #健身
5
#問 健身睡眠
5
健身房的那一群人 #佔用器材 #健身也要健禮貌
5
#問 我想要練出肌肉 但比很多人更累一點。
5
麥當勞x超商x斷食飲食全記錄16+1週EP3 #健身 #減脂 #麥當勞 #TDEE #靈活飲食
5
#分享  動胸肌影片 女孩練胸肌迷思
5
#問 私人教練or健身房？
5
#問 5×5
5
#分享 用體撐訓練胸肌時要避免的錯誤 (中文字幕)
5
#問 大腿內側
5
#問 #減脂#外食
5
#重訓 半年的成長
6
#分享北區運動來團練
6
#問 myprotein 酸酸的
6
#問#健身女孩的保養習慣
6
#問 #重訓
6
#健身#健身工廠#中清廠 徵入會
6
#分享 利用牆壁的伏地挺身動作變化 (中文字幕)
6
#分享 邊聽館長唸你邊健身
6
#圖 我來破解女

10
#問
10
問#第一次
10
#減重 減脂問題+間歇性斷食分享
10
#啞鈴臥推 #自由槓臥推
10
#問 21%體脂 該增肌還是減脂
10
#推薦健身房#健身房
10
#問 #關於減重減脂 有體脂量測結果
10
#問 背肌
10
#減重 間歇性斷食？
10
#請益 健身的最佳時間
10
#問 如何消除壯手臂
10
#有氧  有氧運動的幾個迷思
10
#問 #健身要穿鞋子嗎
10
Inzer 腰帶問題請益 #健身
10
#疑惑：有點尷尬
10
#分享 同時訓練肩膀和胸肌的動作 (中文字幕)
10
#問 減脂運動（求修正）
10
#問 單腳RDL與傳統硬舉
10
#問 想問個一天食量與運動量
10
#身材 腹肌的體現
10
#問 健身時的飲食
10
#圖#myprotein#跪求正確口味名稱
10
#胸肌凹陷
10
#分享 快速燃燒大腿內側的訓練 (中文字幕)
10
#吃飯
10
#台中健身教練 #求推薦
10
#想減脂
10
#問增重買衣服
10
#問 新手用iherb
10
#問 38天瘦了15公斤 我這樣減重正確嗎？
10
#詢問豎脊中立的感覺
10
#問 健身房和戶外運動是兩回事?
10
#問 關於醣類
10
#菜單
10
#問
10
#問 下斜方受傷
10
#分享 手臂訓練所學到的5件事 (中文字幕)
10
#重訓#問
10
#問 #增肌 #糖尿病
10
#健身工廠#續約
10
#問  啞鈴
10
#分享 當你吃了麩質會發生的事 | Thomas DeLauer的健康秘訣 (中文字幕)
10
#健身 有沒有也在練斯巴達障礙賽的？AROO！
10
#問 在健身房跟路人閒聊會很怪嗎
10
#問
10
#問  因為體重關係 想要減重
10
#問 油的熱量計算
10
#問 頭一次健身，注意事項
10
#問 應徵健身房的工作
10
＃心得 #分享 買萵苣的團購券
10
#問 健身時速度問題
10
#問 健身vs運動
10
#問 北高雄短期健身房推薦
10
#問 健工加廠問題
11
#問  中壢附近的健身房
11
#肩膀 該排哪一天
11
#分享 大重量和輕重量的訓練 (中文字幕)
11
#問 健身房
11
#健身房#問#台中
11
#健身
11
world gym價格。#問
11
#問 補給品營養品
11
#分享 WG心得有好有壞
11
#問 人家說重訊時血壓會

#問 莫名其妙就充血
15
#討論 和朋友一起健身比較有動力嗎?
15
#問 器材用途
15
#分享 打造翹臀的訓練動作 (中文字幕)
15
#問 運動後肌肉酸痛 乳清
15
＃問 初心者真的要找教練嗎？
15
#問 關於TDEE
15
#增肌減脂請益
15
#分享 腹肌萬年老常識，死腦筋的還是不懂
15
#分享 補充品做不到的5件事 (中文字幕)
15
#問 重訓時不太流汗
15
#健身 訓練鞋=舉重鞋？
15
#問 健身效果
15
#分享 專家解析 運動員性慾強 關鍵出在賀爾蒙
15
#健身#問
15
#分享 針對手臂和胸肌的訓練 (中文字幕)
15
#疑問   減脂不減肌
15
#重訓 練胸菜單還少了什麼?
15
#問 深蹲硬舉的膝蓋問題
15
#問 大尺寸運動內衣
15
#問 二頭肌
15
#問 新手重訓 請鞭小力一點
15
#問 史密斯臥推
15
#問 感受度與肌酸
15
#問 減重與飲食
15
#健身工廠信用卡
15
#問#高雄健身房
15
#問My protein
15
#問 跑步與深蹲
15
問#動作名稱(抱歉我是超新手)
15
#問#減肥 快走比慢跑好嗎
15
#問題 訓練體能
15
#問 剛接觸深蹲的一些問題
15
#減重 體重一直降不下來qq
15
#分享 我很少推薦的5大訓練技巧 (中文字幕)
15
＃問 要怎樣才知道自己有沒有練到？
15
#健工博愛#揪伴一起練
15
＃問   剛開始槓鈴臥推太重，會影響之後的狀態嗎？
15
#問 各位每次的重訓時間?
15
#問 充滿疑問的心跳率
15
#問 可以分別算出不同克數會有不同G的食物營養素程式或網站
15
#健身 有沒有人在中和健工啊
15
#問私人教練
15
#問 #減肥
15
#問 晨跑想吐反胃
15
#更#減肥餐 請指教
15
#健身 屏東萵苣
15
#問體重數字變少看不出來自己瘦
15
#問 運動
15
#分享 如何伸展你的二頭肌 (中文字幕)
15
#問 改善溜肩
15
#問 每次的暖身
15
#問 板橋樹林有推薦的健身房嗎
15
#問 我想開始減肥
15
#問#增肌速度
15
#問 括約肌怎麼練
15
#分享 10分鐘的壺鈴全身訓練 (中文字幕)
15
#問 教/不教?
15
#問 TDEE 攝取熱量過低一段時間
15
#問 大家對於館長的看法
15
#問 關於有氧及無氧運動的差別

#分享 台中健身房練的超爽
20
#問題 新手發問
20
#分享 8個你從未做過的手臂訓練動作 (中文字幕)
20
#問 教練課程退費及轉讓問題
20
#問 如何準備輕便的運動隨身包
20
#問 游泳後胸口痛
20
#問 活動度 肌力
20
#問椎間盤突出
20
#問 東湖萵苣🙏
20
#問 賀寶芙奶昔
20
#新北物理治療
20
#分享 Zumba Basic1
20
#關於游泳減肥
20
肌肉很好練！？
20
#問 深蹲痠痛
20
#問 深蹲沒力的時候會用腰
20
#減肥 #瘦身 #一個月-17公斤 #我想瘦胸
20
#問 大家有追蹤運動的youtuber嗎
20
#新聞 #問 #練到罩杯縮水？
20
#問 運動隔天酸痛還要繼續嗎？
20
#問 #重訓 我遇到了兩個問題（腿、胸）
20
#問  增加上胸感受度#健身#感受度
20
#健身工廠 合約問題
20
#健身 在健身時請不要放臭氣
20
#問 課表排法
20
#問。健身工廠問題
20
#分享 4個你不應效法的飲食迷思 (中文字幕)
20
#問 運動毛巾
20
＃問 花蓮團體課程
20
#問 背部酸酸的
20
#問 麒麟臂是不是要很低的體脂肪
20
#問 teamjoined台灣官網
20
#問 重訓後補充蛋白質？
20
#問
20
#重訓 腹部旋轉機真的不好嗎？
20
魯夫是8+9的體格！？
20
#問 健身教練應有的實力
20
#問 關於硬舉
20
#問 深蹲 及其他問題
20
#問  練腹肌
20
#問 臥推時使力
20
＃問 Thenx有效嗎
20
#分享 8分鐘的站姿腹部訓練 (中文字幕)
20
#重訓#減肥#問題#女生
20
#問 深蹲架永遠都是滿的
20
#問 價格
20
#運動健身
20
#問 各位的胸肌可以跳嗎？
20
#分享 健身滿一年心路歷程
20
#問 請教運動菜單+如何拒絕推銷
20
#問 腿推舉能暫時代替深蹲嗎?
20
連鎖健身房的商品
20
#重訓 練腿推舉練到股四頭肌三天後還在抽筋
20
#增重型乳清蛋白的問題
20
#問 水梨型身材如何消除？
20
#問 關於飲食
20
#問 Team Joined護膝
20
#問 女生健身菜單
20
#重訓 #問題
20
#討論  乳清蛋白
20
#問 健身房
20
#問 後上背如何瘦
20
#分享 8個會影響訓練成效的睡眠習慣 (中文字幕

...

#健身 #問
34
#減脂 均衡飲食之卡關了怎麼辦?
34
#慢跑#輔大#新莊
34
#乳清分享
34
#健身 關於健身，家人的不理解
34
#健身 增肌的自助餐
34
#健身重訓   乳清的選擇
34
#體脂肪#健身
34
健身房價碼規則
34
#健身 覺得自己力氣很小
不是json
35
#健身 覺得自己力氣很小
35
#更 #健身 如何計算BMR與TDEE
35
#減肥 #詢問有關tdee食物計算
35
#瑜珈 簡易入門學習動作
35
健身新手提問
35
#健身飲食
35
#減肥 我這樣瘦對嗎？
35
#健身#蛋白質來源
35
#重訓 新手同時增肌減脂
35
#請益 推薦可以徵求增肌減脂夥伴的社群
35
#瘦身 我有什麼該注意的
35
#減肥 #詢問 重訓入門
35
健身房揪運動
35
#健身卡
35
#健身卡
35
#瘦身這樣的飲食和運動方式，有機會瘦嗎？求解😭😭😭
35
#紙片人健身增重求解🙏
35
#重訓 健身推薦的書籍
35
#增肌減脂 體重上升
35
#增肌減脂 體重上升
35
#健身 #圖 健身完之後
35
#健身磅數
35
#健身 有一些飲食跟部位上的疑問
35
#問 健身問題
35
#健身 線條不明顯,卡關了QQ
35
#健身 請教減脂 碳水循環飲食問題
35
#健身 關於腰痛疑問
35
飛輪慢慢騎 胸部大減脂(內有噁圖慎入
35
#請益#健身#籃球中鋒訓練
35
#乳清蛋白分析
35
#減重
35
我的身材好嗎？#健身
35
#請益#健身台北推薦TRX課程
35
#健身 乳清蛋白必要嗎？
35
#問 #健身工廠中科廠 #私人教練推薦
35
#健身
35
#健身
35
健身瘦不下來 #健身#瘦身
35
#問 健身房成吉
35
#健身 關於臥推卡關的檢測
35
#健身#問#成吉思漢有意再高雄開廠？
35
#健身 #課表 #解惑
35
#健身 MP高蛋白
35
#健身 從小胖到大到接觸運動的改變! (圖)
35
#健身    健身身身（瘦）
35
#術後 #瘦身運動
35
#關於游泳 減脂
35
#紙片人高蛋白選擇
35
#減肥 #飲食 #瘦身 晚餐改吃大燕麥片?!
35
#是否需用乳清
35
#減肥 #慢跑 小腿緊到快死了 第一次這樣
35
#重訓 #高蛋白
35
#健身 #器材
35
#減脂 覺得肚子的皮有點鬆
35
#健身 Myprotein

久久沒健身，最近要回鍋請教
39
#健身#有無不重訓長胸肌的八卦
39
#黑特 #健身房 丟臉
39
#圖#第一次的夜跑😂
39
健身吃牛肉比雞肉好？
39
#健身 怎樣才算好的健身教練？
40
#圖  乳清請益
40
#聊 夜跑/晨跑
40
#健身 請益增加力量
40
#健身 #減肥 要買高蛋白(乳清)的朋友快來喔!
40
#減脂  飲食方面請教
40
#問 大里霧峰健身房
40
增肌減脂停滯期？
40
#健身 給新手健身費用之參考
40
吃不胖 健身
40
#健身 深蹲腰酸是姿勢錯誤嗎？
40
#補圖 其實瘦身沒有你想像中這麼困難。
40
用嘴巴健身
40
#健身 啞鈴器材
40
#world gym 揪運動健身~
40
健身房難抉擇！
40
#健身 健工信義
40
請益減重問題
40
#健身 硬舉&睡眠問題
40
#跑鞋 大家慢跑習慣買什麼牌子的鞋子
40
90公斤~89公斤的健身之路
40
#問#永和健身院 楊雄
40
低GI飲食搭配運動 讓你減脂事半功倍
40
健康生活／健身很久都沒進步 你的睡眠習慣可能需要調整
40
#健身 想請問乳清挑選
40
#健身 沒有最扯只有更扯
40
#健身 【請益】蛋白質補充不夠重訓是否沒用？
40
Re:#圖 健身V.S不健身 完全是兩種不同人生！
40
我該花錢上健身房嗎？
40
#健身#轉載 給想要踏入健身房的你的幾個指引
40
#健身 重訓彈跳力下降問題
40
#問 台灣健身重量單位？
40
#問 有沒有辦法能瘦胸，健身有用嗎？
40
#慢跑 疼痛狀況正常嗎
40
請健身教練課程為什麼這麼貴？
40
#慢跑 #半馬 請教從零到半馬的十一週規劃
40
#健身 在家只有啞鈴想健身
40
#慢跑 #問 各位剛開始慢跑時
40
#問 尋找健身房教練
40
#問 靠什麼運動瘦身最好
40
#問 健身房
40
#問 高強度間歇訓練真的對減脂無效嗎
40
#圖  健身V.S不健身 完全是兩種不同人生！
40
慢跑Day4
40
#健身 Roadblock 備賽分享&你們想看的是什麼！？
40
#健身 關於減脂時期的水果
40
#慢跑 脊椎側彎適合跑步嗎？
40
#健身  拉力帶 助力帶 使用最佳時機
40
#減脂 錯誤觀念
40
慢跑Day3
40
#徵人 健身工廠 精明
40
#健身 關於器材品牌疑惑
40
#問 減肥菜單
40

慢跑大家都聽什麼音樂 & 下雨不能跑步怎麼辦呢？
44
健身上的小問題大事情
44
#《阿屁》健身 可能跟你想的不一樣 ─ 關於訓練頻率與組數
44
#健身 疑問 關於此（爆發力）彈跳訓練
44
#問 慢跑後拉筋膝蓋痛
44
#問 關於減脂的日常小問題
44
#問  推薦的健身女教練
44
＃健身 看到一個台灣的健身品牌
44
慢跑左腹會痛
44
想請問下各位對全真瑜珈健身的評價跟看法🌞
44
淡水健身工廠
44
#健身 Roadblock 新菜單週期嘗試
45
#健身 練馬甲線
45
#慢跑 #問 肌貼
45
在健身房裡被男生糾正動作
45
減肥好難啊…
45
#問#重訓 有氧 飲食 減肥
45
請問健身飲食如何搭配
45
【問卷: 贈送50P幣】使用「計次收費」健身房的同學請進
45
好想找個人帶我健身
45
#健身 尋找屏東夥伴
45
#運動、減肥飲食控制發問
45
（有氧）如何當上健身房的有氧老師
45
#問 慢跑腳背痛
45
想問減肥瘦身吃法
45
胖仔想瘦身
45
#健身  做了inbody以後，感覺不準
45
#健身 深蹲上背不舒服
45
#健身瘦身想尋求建議
45
#健身 增肌
45
#健身 圖《阿屁》  Re: 健身新手 卡關疑問
45
#分享 運動時，先肌力還是先有氧？
45
#健身 請問林口有健身房嗎？
45
#運動減肥飲食
45
#健身 同學都花錢了 就不要一直低頭了吧
45
健身完翻船 頭暈 看不見
45
健身新手 卡關疑問
45
#健身 （更新）觀念淺談-週期化訓練
45
#減重 問肌肉型降體重
45
#減肥 #減重 讓我告訴你    為甚麼瘦不下來
45
#減重  3天斷糖?!
45
#健身 觀念淺談-歡迎討論
45
#健身#圖 關於喝完高蛋白脹氣
45
#健身相關 各位行行好，幫忙填寫一下問卷
45
#健身 #討論 淺談證照 教練體態 以及選擇(長文)
45
#減肥 #重訓 大家問到爛卻不學的問題
45
請問有在健身的大大，上胸肌跟下腹肌怎麼練比較好？
45
#健身 #心得 M.E.T 動作效益訓練
45
問資歷就惱怒的健身教練？
45
#健身 腹肌線條不明顯
45
#健身 人中之龍的條件達成難易度?
45
#重量訓練 Re: #健身 放槓&摔槓
45
#減肥 一週-2kg，會不會是餓瘦的？！
45
#健身 放槓&摔槓
45
[問］N

#健身 初接觸，該如何讓背闊肌更吃力
49
#減重  各位可以給我建議瘦身嗎...?
49
#健身# 德國壯漢
49
#健身 想請問啞鈴哪裡有在賣
49
#瑜珈 如何透過瑜珈或任何運動矯正大轉子突出呢?
49
#健身 變大變久
49
#健身  有氧重訓時間分配
49
#瘦身 要怎麼瘦小腿肌肉
49
#健身  要怎麼健身?
49
#健身 tabata真有這麼神嗎？
49
#瘦身 運動型的肌肉怎麼消?
49
#健身  高蛋白詢問
49
#健身 沒有感覺的動作，還有效果嗎
49
#健身 關於健身的飲食建議 求建議搭配
49
#減重 不吃澱粉是對的嗎
49
#瘦身 仰臥起坐板弧形和直的，哪個比較好？
49
#健身 關於健腹器
49
#瘦身  瘦到肌肉很悲劇
49
#瘦身 關於瘦臉
49
#搏擊 健身與格鬥---68公斤柔術家對上113公斤的健美選手
49
#健身 重訓問題
49
#慢跑 慢跑讓經期延後了?!
49
#健身 #美體 如何消副乳
49
#健身 乳清蛋白 肚子餓
49
#健身  男性女乳症怎麼辦QQ
49
#健身 #美體 #減肥 運動迷思與減肥迷思
49
#健身 小腿
49
#健身 運動耳機
49
#瘦身 關於大腿與小腿
49
#健身 引體向上
49
#瘦身  什麼才算有運動到 ？
49
#瘦身 這樣的飲食搭配運動是否OK
49
#健身 健身飲食
49
#健身 想問有沒有一個數字的代表人物
49
#健身 肌肉好像不太發達><
49
＃瘦身 我已經快走投無路了拜託大家救救胖到一無是處的我
49
#健身 有人喝過惡魔基地的乳清嗎?
49
#健身 腹肌抽筋
49
#健身  健身與我
49
#健身 拮抗
49
#瘦身 關於瘦臉
49
#慢跑 運動新手
49
#瘦身 請問跳鄭多燕+肌力運動可以幫助瘦腿嗎
49
# 健身 核心肌群訓練應該要多久才對?
49
#健身 鐵腿=乳酸堆積？迷思
49
#健身 大家到健身房做的第一個項目是??
49
#瘦身 跳舞跳到菊花長痘痘!!!
49
#瘦身  關於小腹
49
#健身 TABATA新手，並詢問塑胸腹背心
49
#瘦身  關於停滯期這件事...
49
#健身 鐘國哥來分享自己的背肌鍛鍊
49
#健身 深蹲跟小腹
49
#減重 一定要吃到基代阿！！
49
#減重       RE: 打臉 女孩板上臉書的【10天瘦了三公斤 我辦到了 】


,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,232227133,如何練成歐美肥臀？,用台灣的健身網紅Ashlee與Jessica舉例\n我一直好奇為什麼兩位體型不太一樣\n我想...,用台灣的健身網紅Ashlee與Jessica舉例，我一直好奇為什麼兩位體型不太一樣，我想過以...,2019-10-04T03:25:25.304Z,2019-10-04T03:25:25.304Z,5,健身,fitness,F,1,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 肌肉, 體脂, 飲食, 健身]"
1,232227083,啞鈴划船問題,各位巨巨好，想問在做單手啞鈴划船時，採用弓箭步，將重量重心放在前側腿及屁股，背是平的，疑問在...,各位巨巨好，想問在做單手啞鈴划船時，採用弓箭步，將重量重心放在前側腿及屁股，背是平的，疑問在...,2019-10-04T03:13:28.850Z,2019-10-04T03:13:56.019Z,2,健身,fitness,M,1,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[重訓]
2,232226994,老人根本是健身房的毒瘤,最近公立運動中心健身房越來越多老人\n尤其是非熱門時段\n他們總是插銷插最輕\n用完全錯誤的...,最近公立運動中心健身房越來越多老人，尤其是非熱門時段，他們總是插銷插最輕，用完全錯誤的姿勢做...,2019-10-04T02:54:57.521Z,2019-10-04T02:54:57.521Z,8,健身,fitness,M,5,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[健身房, 健身器材]"
3,232226923,inbody 名詞/測量項目請益,https://i.imgur.com/12XHp7o.jpg\n\n各位巨巨好，小的新手健...,各位巨巨好，小的新手健人，接觸約2個半月，第一次量inbody，因為一些緣故所以沒有專業的人...,2019-10-04T02:36:10.001Z,2019-10-04T02:37:20.724Z,0,健身,fitness,M,0,[],"[Inbody, 健身]"
4,232226709,睡眠很重要 不要沒睡飽,奉勸各位如果前一天沒有睡好\n就盡量不要去做重訓\n當然這是我個人的經驗談\n不一定每個人都...,奉勸各位如果前一天沒有睡好，就盡量不要去做重訓，當然這是我個人的經驗談，不一定每個人都適用 ...,2019-10-04T01:48:18.828Z,2019-10-04T01:50:23.495Z,5,健身,fitness,M,5,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[重訓, 健身房]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11771,187976,健身問題：腹肌,小弟我175cm 63kg 體脂肪9趴\n算是有點偏瘦 在努力吃胖中：3\n有明顯的胸肌跟背...,小弟我175cm 63kg 體脂肪9趴\n算是有點偏瘦 在努力吃胖中：3\n有明顯的胸肌跟背...,2015-05-11T07:30:01.705Z,2018-03-06T11:33:29.322Z,25,健身,fitness,M,22,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[腹肌]
11772,106299,徵求:各式各樣關於健身的問題,大家好!\n\n由於我最近要在校內為社團講課，主題是「基礎健身/重量訓練」\n目前我計畫講解...,大家好!\n\n由於我最近要在校內為社團講課，主題是「基礎健身/重量訓練」\n目前我計畫講解...,2015-03-17T13:28:53.168Z,2015-03-17T13:28:53.168Z,13,健身,fitness,M,9,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[]
11773,100157,【問】嘿 關於健身,這邊應該有很多30cm有研究過健身這檔事\n小魯弟來發問一下\n天天都慢跑是不是真的有助於瘦...,這邊應該有很多30cm有研究過健身這檔事\n小魯弟來發問一下\n天天都慢跑是不是真的有助於瘦...,2015-03-10T11:52:32.333Z,2015-03-10T11:52:32.333Z,16,健身,fitness,M,2,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[]
11774,99857,健身提問-關於乳清蛋白,各位卡友大家好\n小弟接觸健身一年多\n去年六七月跟PTT上的板友們一起團購BSN的乳清蛋白...,各位卡友大家好\n小弟接觸健身一年多\n去年六七月跟PTT上的板友們一起團購BSN的乳清蛋白...,2015-03-10T03:25:59.121Z,2015-03-10T03:25:59.121Z,5,健身,fitness,M,16,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[]


In [44]:
# 將資料存檔
df.to_excel('E:/Python 3.7/pyetl/Demodb0103/gym/Dcard_1.xlsx', engine='xlsxwriter')

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://i.imgur.com/12XHp7o.jpg

各位巨巨好，小的新手健人，接觸約2個半月
第一次量inbody，因為一些緣故所以沒有專業的人在旁邊詳細解釋，所以有些細節不是很懂。

正題~

有疑惑的點是..%20%20%20
我的骨骼肌測出來約29kg

再來部位別肌肉分析那欄有看沒有懂
各部位重量怎麼相加相減都跟29kg不對盤...
以及下方的%25數也是一知半解..

請問各位大大知道為何兩者之間(骨骼肌和各部位肌肉)不相等嗎？
或者兩項測量的東西各是什麼？

最後
右邊小欄的部位別脂肪分析中的200%25上下的數字又代表什麼意思呢？

問題整理：
1.各部位肌肉加總為何大於骨骼肌重量？多的是什麼？
2.各部位肌肉分析中的兩項數值各是什麼？
3.各部位脂肪分析中的%25數值代表什麼意思？
感謝各位大大了%20%3e_%3c' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://giftbox.dcard.tw/groupBuy/5d8c395e6f6749e8f22edad4/header?isWebView=true
工作日天天出貨

👍🏻👍🏻%20app%20右下角有立即購買，沒有的話記得更新%20Dcard%20app，你可以開手機瀏覽器或是桌機上直接買也可以喔👍🏻👍🏻
⚠️透過卡友信內的『訂單進度查詢』商品貨態，如逾期未領須負擔運費重寄或主動申請退款
⚠️冷凍食品，本次只提供宅配服務喔！

卡友價：
🥩迷迭香肩舒肥牛排10入%20售價%20NTD%20$%202399%20元
🥩UP低溫嫩雞即食包25入%20售價%20NTD%

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://i.imgur.com/7udJY68.jpg
這照片紀錄了從18歲～20歲的變化
體重從67公斤➡️82公斤➡️69公斤
因為在高中訓練時脊椎受了傷導致不能繼續訓練
但食量還是很大沒在控制瘋狂的亂吃%20
不到半年體重直接從67公斤飆到82公斤
短短的兩年讓我的身體從瘦到胖再變現在這樣

第一張照片：
是我高中體育班的時候
因為有接觸重訓所以身材還不算太差
那時候體重都維持在67KG
有些人會覺得第一張跟第四張看起來差不多
但不管厚度寬度肌肉大小就差很多
只是那時候角度抓得好%20照騙了

第二張照片：
這張照片是去滑完龍舟坐著放空的時候
我同學覺得太胖就偷拍了下來%20
一覽無遺的幫我拍下人生中最肥的照片
畢竟我那時候是完全不運動的狀態
還有一張我們全家出去玩拍的
超扯的%20胖成這樣真的無法接受
https://i.imgur.com/vJaBWVF.jpg
就是這時候體重來到了82公斤
為什麼我開始意識到自己胖了
因為上下樓梯還是走路%20開始容易喘
身邊的家人朋友同學開始
從「壯」改變為「胖」、「肥」這個形容詞
膝蓋常常不舒服，坐下來肚子擠成一坨
衣服還會凸出來🤦‍♂️

第三張照片：
開始重新去運動去健身%20飲食控制
就有稍微瘦下來一點就77公斤了
其實我那時候就是在萵苣當教練
工作時常很忙最後又中斷了
所以就一直維持著不明顯的肌肉線條
最後覺得健身教練還是要有健身教練的體態
還有一些原因我就毅然決然的離職又休學
報名了大陸的健身學院去做體態改造訓練
這是3月出國前的照片

第四張照片：
當然有了老師、環境、時間更容易讓自己改變
但因為許許多多因素不能全歸功於學院
是靠著自己努力以及同伴們的輔助與鼓勵下成長
結束回國後雖然是變壯%20，但體脂還是偏高
剛好在大陸訓練時報了第一場比賽
每天把健身房當家一樣去%20規劃訓練課表和飲食
這段時間的付出%20真的是很多的努力跟堅持
才讓自己重新達到自己喜歡的樣子甚至更好

⬇️這是去比賽的照片
https://i.imgur.com/2JaPiwu.jpg

⬇️這是比完賽的隔天

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://bitty.tw/louisa-coffee-calories/%20

我參考這個網站%20看%20%22全麥烤腿排%22%20大概是%20413大卡
可是網站的版主實測%20又寫291大卡

到底哪個是對的?
最近都吃全麥烤腿排%20或者%20全麥鮪魚三明治%20當作早餐跟晚餐
然後各配一杯乳清

早餐
乳清;140大卡
烤腿排;415大卡%20%20(大概)

中餐
家裡自備便當;800大卡%20%20(大概)

晚餐
乳清;140大卡
烤腿排;415大卡%20%20(大概)

這樣是1910大卡
我的TDEE%20是%202300
一周健身五天

總結:
針對該吃甚麼
我想問的是=%3e%20吃路易莎%20%20全麥鮪魚%20%20或者%20%20%20全麥烤腿排%20%20%20%20來做減脂%20%20有沒有問題?

謝謝大家' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://www.dcard.tw/f/fitness/p/231704587?ref=ios
⬆️上一篇發文本來想連飲食都一起分享，但實在太攏長所以就再寫一篇與大家分享～

必須說去年未開始真正碰健身和正確減肥前，我真的不知道什麼是攝取「好的碳水」、「好的脂肪」，也不知道補充蛋白質的重要性%20，只知道吃自己想吃的，有吃到飯菜肉跟蛋好像就很營養健康了

之後找了教練除了有正確的健身觀念以外，對飲食這部分也做了很大的改變，不管事身理還是心理我真的都覺得改善很多

以前的三餐就是想吃什麼就買什麼，家裡煮什麼就吃，白飯都要吃到第二碗，而且都是添好添滿的那種境界，上班族

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/AWC01kH.jpg
這本書以背蹲舉、臥推、硬舉、上推、上搏（抓舉）為主軸，那在書裡講解的非常仔細，為什麼做、如何做、其實很少書籍把一個背蹲舉的細節講的那麼詳細的（確實背蹲舉是有很多細節的），那在書籍後面有延伸的補強動作，課表安排（訓練與恢復的安排），如果讀者領悟力夠好，確實初學者可以透過自學來讓自已變強壯，只是需要知識背景，不然看這本書可能會很難過。

更:博客來網站上有試閱可以先看看內容
%20https://www.books.com.tw/products/0010821384' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/PBcDuQr.jpg

http://bit.ly/31VV0Sq

這次的內容針對屁股眨眼詳細說明，為了避免深蹲時的屁股眨眼，建議先調整深蹲的深度，並檢視自己是否有以下的問題逐一調整，再追求深蹲時的深度。

1.髖關節囊的活動度問題
2.髖關節結構的限制
3.骨盆肌肉的過度緊繃
4.腳踝的活動度和柔軟度的問題
5.骨盆穩定度的問題

%20
--
Fitting%20Room%20TW
運動、健康相關資訊翻譯影片
官方Blog
http://fittingroomtw.blogspot.tw/
官方臉書
https://goo.gl/HZhAhS' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url)

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://i.imgur.com/yEA2Ecn.png
https://i.imgur.com/n7vfMjd.png
https://i.imgur.com/GhYLxzY.png
https://i.imgur.com/Btz7p3s.png
https://i.imgur.com/WXSfV2m.jpg

誰要PDF檔可以跟我說

閒閒沒是和作者合作
他同意我免費給中文地區60頁免費的電子書
https://docs.google.com/forms/d/e/1FAIpQLSdBdtBcxzmirt2vcZq98uQNUWR7TW2SBKXctXlR5L-qxW8KLw/viewform?usp=sf_link
感謝' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/8dfWBOF.jpg

http://bit.ly/2JaWDV9

%20這次的內容是介紹做過頭肩推前的暖身流程，當你要訓練肩膀時，感覺肩膀的活動度不夠，這時可能是因為沒有啟動到正確的肌肉，可以拿一顆抗力球以跪姿面對牆壁，這時用手托住球的下方以手腕發力將球沿著牆面往上推，推的力道不用太大，不要讓手指碰觸到抗力球，單邊重複6到7下之後換到中間，重複操作之後再換另一邊，藉此來啟動可以幫助雙手高舉過頭的肌肉，藉此改善肩推時的活動度。%20

%20
--
Fitting%20Room%20TW
運動、健康相關資訊翻譯影片
官方Blog
http://fittingroomtw.blogspot.tw/
官方臉書
https://goo.gl/HZhAhS' 

...

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/t35EA0s.jpg

%20https://goo.gl/ES8xTH%20
%20
關於減脂的方法，我們在日常生活中可能會聽到各式各樣的建議，飲食方面，像是地瓜有著相當高的營養價值，而糙米飯比起白飯更健康，或者是關於運動的建議，像是空腹的有氧運動更能幫助減脂，然而這些建議未必能夠有所幫助，如果不適用在我們身上，這些都不是好的建議。

很多人在營養攝取方面會仰賴補充品，甚至是用補充品取代正餐，補充品雖然有所幫助，但無法為你建立正確的飲食觀念，長期累積也是額外的開銷，因此將補充品作為輔助，逐漸改善飲食習慣才是長久之計。

對於減重和體態調整，過程中保持一致性很重要，如果欺騙自己就會影響到成效，但很多人因為無法維持一致性，無法開始做出改變，或是在改變途中選擇放棄，因此影片中的建議，是逐漸做出改變，藉由改變的結果再激勵自己付出更多的努力，如果一開始設下過於嚴苛的目標，許多人反而會選擇不去改變，最後只能維持現狀，然而每個人的狀況都有所不同，不管是飲食習慣或是運動能力，根據自己的狀態來逐漸調整，才能夠真正長期維持。
%20
--
Fitting%20Room%20TW
運動、健康相關資訊翻譯影片
官方Blog
http://fittingroomtw.blogspot.tw/
官方臉書
https://goo.gl/HZhAhS' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/H1ahUP7.jpg

%20https://goo.gl/zYGGcv%20

%20這次所設計的高強度間歇訓練，是利用

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/GrXeA9W.jpg

%20https://goo.gl/CrGvZL%20

很多人因為不了解下背痛的成因，當有下背痛的症狀出現時只會針對痛的位置處理，但長久下來還是無法克服這樣的症狀，而飽受病痛之苦。

下背痛的成因主要是因為下方的髖關節和臀肌疏於鍛鍊，讓身體用下背完成許多動作，長期下來就造成了下背痛的問題，為了避免這個狀況，要先熟悉髖關節鉸鏈的動作，避免用脊椎屈曲。

可以將彈力帶固定在深蹲架上，調整到髖關節的高度，並以一腳在前一腳在後的狀態，讓身體往前承受阻力，再往下做髖關節鉸鏈，藉此鍛鍊臀肌和髖關節。

如果背部健康，但想要加強臀肌和髖關節的鍛鍊，可以在彈力帶的限制下利用羅馬尼亞硬舉，這樣在舉起槓鈴時，臀肌仍要承受彈力帶的阻力，藉此來加強。

對於想要更進階的人，可以在羅馬尼亞硬舉的動作中再加入屈體划船，這樣的動作模式會讓腰椎、胸椎、髖關節需要一起合作，更進一步提升訓練強度。
%20
--
Fitting%20Room%20TW
運動、健康相關資訊翻譯影片
官方Blog
http://fittingroomtw.blogspot.tw/
官方臉書
https://goo.gl/HZhAhS' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/3rWlVJr.jpg

%20https://goo.gl/oKX6EV%20

%20
這次是利用藥球來針對腹部的訓練，動作包含負重仰臥搖擺、卷腹過頭上拋、俄式砸球、三方向的砸球、三段卷腹，藥球的重量不重要，應追求動作的標準。

仰臥搖擺：如

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/hnoXVFj.jpg

https://goo.gl/XQxC2Q

今天翻譯的影片，是我取自ATHLEAN-X頻道，所製作的第80部中文字幕影片，考慮到原頻道目前是795部影片，等於ATHLEAN-X的內容現在有10%25已經有中文的版本，算是一個小小的里程碑，如果可以的話，如果你曾經因為我翻譯的影片學到一點東西，或是得到幫助，希望大家去Jeff的原頻道點訂閱，以及他的其他社群媒體，表達對他的支持，感謝大家%20:)

ATHLEAN-X%20Youtube頻道
https://www.youtube.com/user/JDCav24

ATHLEAN-X%20Facebook頁面
https://www.facebook.com/athleanx

ATHLEAN-X%20Instagram頁面
http://instagram.com/athleanx

--
Fitting%20Room%20TW
運動、健康相關資訊翻譯影片
官方Blog
http://fittingroomtw.blogspot.tw/
官方臉書
https://goo.gl/HZhAhS' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://i.imgur.com/cOlRfyz.jpg
不好意思想請教一下版上各位前輩
這是小弟昨天去健工所測量的inbody
由於是在晚上量的可能還是會有落差

開始運動的原因是因為
退伍的時候發胖很多
想說這樣下去不行所以靠著節食與每天慢跑
由於當時還沒有什麼觀念只是想瘦
所以就這樣從78kg掉到了72kg

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://youtu.be/WubdKkjaLr8

幫大家整理重點
1.並沒有所謂燃燒脂肪這件事
2.體脂下降的路徑是，肌肉細胞跟脂肪細胞搶養分，肌肉細胞搶得越多，脂肪細胞就不易增長。
3.對減少脂肪而言無氧運動比有氧運動更好
4.運動完立即吃飯有助於減肥(並不是額外攝取食物，是指要你在原本就要吃的正餐前運動)
5.飲食的影響很大
(這個影片沒有提到，但是在其他醫學報告中，單就「體重」而言，飲食與體重%20相關程度超過80%25，遠高於運動比例)

但是這位教授並沒有否定有氧運動，有氧運動有很多其他的有點，譬如，提高心肺能力，降低心血管疾病風險等。這篇文章只是單就減脂肪去做論述。

其實國外在十年前，已經很流行這種說法，無氧可以控制脂肪，但是在台灣這幾年還是流行有氧運動的燃脂效果。

按照此位教授的說法，以下大家從小聽到大的論述都是錯的
1.運動三十分鐘以上，可以有效燃燒脂肪
2.運動後吸收比較好，不要吃東西
等' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://i.imgur.com/dzrbOSm.jpg
👆🏻健身初期


我的故事：
以前嚮往著想變成猛男，卻不知如何開始，
從升大三那個暑假，發現身邊有個朋友越來越壯，因為學校附近沒有大型健身房，只能到雲林健力室，器材不多但要自己去找不同變化動作，曾經的我是不論怎麼吃都很難胖就保持健身初期的樣子，打工、讀書、運動忙錄，一天三餐不正常暴飲暴食，愛吃宵夜，又熬夜喝酒！發現自己還是瘦到一個不行，精神體力都不是很好，透均衡的營養，充足的水分，總共增重10公斤！目前開始減脂在兩個月減了7公斤！

⚠️


E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/UoW02YG.jpg
這是我每天跑步40分鐘消耗的卡路里。
前幾天跑完大概會吃1顆蓮霧+10個小番茄
剛剛在網路上看到要適當補充蛋白質與碳水化合物，想請問：

1.跑完步、打完球可以喝牛奶嗎？喝多少才適量？要喝低脂還是一般的牛奶？
2.碳水化合物可以吃白吐司嗎？或是有推薦吃什麼方便取得的東西嗎？
3.以上東西應該馬上吃？還是要間隔多久在吃？

因為本身還會做點伏地挺身、仰臥起坐，修飾一下線條，以上物品應該要分兩次吃？或是全部的運動做完一起吃呢？

3.運動完的時間大約都在晚上12點左右，這時間吃會不會造成肥胖？' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/uQHiPC0.jpg
目前我看最便宜的%20算一份31.2元左右%20%20總價2398
應該是這樣算吧%20總共77份
　　　　%20%20%20%20%20%20　每份　　　
熱量　　　120大卡
蛋白質　　24公克　　　
脂肪　　　1公克　　%20%20%20%20%20%20%20%20%20%20%20%20%20
　飽合脂肪0.5公克　　%20%20%20%20%20%20
　反式脂肪0公克　　　
碳水化合物3公克　　%20%20%20%20
　糖　　　1公克　　　%20%20
鈉　　　　130毫克　


還是大家有其他好品牌%20或同ON品牌%20別款%20可貼圖%20跟貼價錢%20謝謝
http://i.imgur.com/zYNpffR.jpg
55份%20我算是1份%2034.5元
　　　　%20%20%20%20